# A Model for the Human Chromosome
This example shows a simple simulation of a single block copolymer that is a potential model of the Human Chromosome in hoomdd.

The human genome consists of 24 chromosomes with a total size of 3Gb.  Aiden et al. 2014 produced a 1kB resolution (below single gene resolution) Genome Contact Map.  Contact domains (∼185 kb) segregate into six nuclear subcompartments with distinct histone marks.
This can be reproduced phenomenologically with this simple block copolymer model.

from: https://lost-contact.mit.edu/afs//umich.edu/user/j/o/joaander/Public/hoomd-web/doc/page_example_scripts.html

## Initialize
First import hoomd and associated libraries and then initialize.  If no mode is given then the fastest processor cpu or gpu will be selected automatically.

In [1]:
#import hoomd
import hoomd.md
from hoomd import *
from hoomd.deprecated import *
#context.initialize("--mode=cpu");
context.initialize("--mode=gpu");

HOOMD-blue v2.2.2-29-g1e86eec CUDA (8.0) DOUBLE HPMC_MIXED MPI SSE SSE2 
Compiled: 01/29/18
Copyright 2009-2017 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
notice(2): This system is not compute exclusive, using local rank to select GPUs
notice(2): Unable to identify node local rank information
notice(2): Using global rank to select GPUs
HOOMD-blue is running on the following GPU(s):
 [0]  GeForce GTX 1060 6GB  10 SM_6.1 @ 1.84 GHz, 6075 MiB DRAM, DIS


## Import other libraries and define parameters

The human genome has 24 chromasomes but for now we will simulate a single chromosome.

In [2]:
import math
import numpy as np
# parameters
phi_P = 0.25
n_poly = 1

## Define the polymer
The average length of a human chromosome is 133 Mb

In [3]:
genome_size=3079843747
Nchromosomes=24 #Mb
Nbeads=genome_size/10000
#chromosome=np.linspace(1,Nchromosomes,Nchromosomes,dtype=intp).tolist()
chromosome_size=np.fix(np.r_[8,7.9,6.5,6.2,5.9,5.5,5.2,4.7,4.6,4.4,4.4,4.3,3.7,3.5,3.3,2.9,2.6,2.5,2.1,2.0,1.5,1.6,5.0,1.9]/100.0*Nbeads)
chromosome=list('bcdefghijklmnopqurstuvwxy')
print("["+"]*%d+[".join(map(str, chromosome))+"]*%d")
print([val for pair in zip(chromosome, chromosome_size.astype(int)) for val in pair])

[b]*%d+[c]*%d+[d]*%d+[e]*%d+[f]*%d+[g]*%d+[h]*%d+[i]*%d+[j]*%d+[k]*%d+[l]*%d+[m]*%d+[n]*%d+[o]*%d+[p]*%d+[q]*%d+[u]*%d+[r]*%d+[s]*%d+[t]*%d+[u]*%d+[v]*%d+[w]*%d+[x]*%d+[y]*%d
['b', 24638, 'c', 24330, 'd', 20018, 'e', 19095, 'f', 18171, 'g', 16939, 'h', 16015, 'i', 14475, 'j', 14167, 'k', 13551, 'l', 13551, 'm', 13243, 'n', 11395, 'o', 10779, 'p', 10163, 'q', 8931, 'u', 8007, 'r', 7699, 's', 6467, 't', 6159, 'u', 4619, 'v', 4927, 'w', 15399, 'x', 5851]


So if we assume that Euchromatin makes up 60% of the genome and heterochromatin makes up 40% then a block copolymer of 10000 beads would have a resolution of approximately 13 kb/bead the average size of a human gene.  This would correspond to approximately 18 beads per enhancer promoter loop.

In [4]:
#type_string=dict()
#polymer1 = dict(bond_len=1.2, type=,bond="linear", count=Nchromosomes)
polymer1 = dict(bond_len=1.2, type=['b']*300 + ['c']*400 + ['b']*300,bond="linear", count=n_poly)
# perform some simple math to find the length of the box
N = len(polymer1['type']) * polymer1['count']
# generate the polymer system
init.create_random_polymers(box=data.boxdim(volume=10*math.pi * N / (6.0 * phi_P)), polymers=[polymer1],separation=dict(b=0.35, c=0.35),seed=12)


notice(2): Group "all" created containing 1000 particles


## Setup the bonds and force fields

In [5]:
# force field setup
harmonic = hoomd.md.bond.harmonic()
harmonic.bond_coeff.set('polymer', k=330.0, r0=0.84)
nl = hoomd.md.nlist.cell();
lj = hoomd.md.pair.lj(nlist=nl, r_cut=3.0)
lj.pair_coeff.set('b', 'b', epsilon=1.0, sigma=1.0, alpha=0.0, r_cut=2**(1.0/6.0))
lj.pair_coeff.set('b', 'c', epsilon=1.0, sigma=1.0, alpha=0.0, r_cut=2**(1.0/6.0))
lj.pair_coeff.set('c', 'c', epsilon=1.0, sigma=1.0, alpha=1.0)
lj.set_params(mode='shift')
                  


## Special Pairs will create our Contact Domains (not working yet)

In [6]:
#CTCF = hoomd.md.special_pair()
#lj_ctcf = hoomd.md.special_pair.lj(name="CTCF")
#lj_ctcf.pair_coeff.set('A','A', epsilon=1.0, sigma=1.0, r_cut=3)
#lj_ctcf.set_params(mode='shift')

## Integrate the simulation

In [7]:
all = group.all()
# integrate NVT for a bunch of time steps
hoomd.md.integrate.mode_standard(dt=0.005)
hoomd.md.integrate.nvt(group=all, kT=1.2, tau=0.5)
#hoomd.md.integrate.brownian(group=all, kT=1.2, seed=1)

The results of this example may be visualized with VMD which can be installed from here: http://www.ks.uiuc.edu/Research/vmd/

In [8]:
#hoomd.dump.gsd(filename="hum_gen_24x1000.gsd", overwrite=True, period=None, group=group.all(), time_step=0)
hoomd.dump.gsd("gsd/24xchromosomes_10000_0.gsd", period=None, group=all, overwrite=True, time_step=0);
# setup the IMD server
hoomd.analyze.imd(port=54321, period=100)
# run a very long time so the simulation can be watched in VMD
run(1e6)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 1 exclusions             : 2
notice(2): Particles with 2 exclusions             : 998
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no
** starting run **
notice(2): analyze.imd: listening on port 54321
Time 00:00:10 | Step 87362 / 1000000 | TPS 8736.16 | ETA 00:01:44
notice(2): analyze.imd: accepted connection
Time 00:00:20 | Step 172601 / 1000000 | TPS 8523.89 | ETA 00:01:37
Time 00:00:30 | Step 254955 / 1000000 | TPS 8235.38 | ETA 00:01:30
Time 00:00:40 | Step 334753 / 1000000 | TPS 7979.78 | ETA 00:01:23
Time 00:00:50 | Step 417526 / 1000000 | TPS 8277.18 | ETA 00:01:10
Time 00:01:00 | Step 507377 / 1000000 | TPS 8985.02 | ETA 00:00:54
Time 00:01:10 | Step 594651 / 1000000 | TPS 8727.32 | ETA 00:00:46
Time 00:01:20 | Step 684422 / 1000000 | TPS 8977.03 | ETA 00:00:35
Time 00:01:30 | Step 773940 / 1000000 | TPS 8951.64 | ETA 00:00:25
Time 00:01

The results of this example may be visualized with VMD which can be installed from here: http://www.ks.uiuc.edu/Research/vmd/

While running must execute the following command in a terminal:

In [9]:
#vmd -e imd6.vmd
!cat imd6.vmd

mol load gsd gsd/24xchromosomes_10000_0.gsd
#set sel [atomselect top all]
#$sel set radius 0
imd connect localhost 54321


![](snapshots/chromosome_10kb.png)